# Train DeepGrow for AIAA


The most important data points to enhance your model are the patients where the model performs badly. 
That means the AIAA segmentation model would fail. 
The fail back here is to ask radiologist to annotate/ fix errors with manual tools. 
This is very time consuming and frustrating. 
Instead we can train a deep grow in 2D or 3D to guid your model to the area where it should segment. 
This requires us to train a different model which is deep grow. 
However, the good news is this uses the same data with the same format you already have.
<br><img src="screenShots/AIAADeepGrow.png" alt="Drawing" style="height: 400px;"/><br>

In this notebook, we will walk you through training deep grow models to be used with AIAA. 

This notebook will work through the following steps:<br>
1. Train a 2D deep grow model then deploying it on AIAA server.
2. Train a 3D deep grow model then deploying it on AIAA server. 


## Prerequisites
- Familiarity with Clara train and MMAR 
- Ran step 9 in [Download Decathlon DataSet Notebook](../Data/DownloadDecathlonDataSet.ipynb) to download the spleen dataset.


## DataSet 
The spleen segmentation task in this notebook performs volumetric (3D) segmentation 
of the spleen from CT images. The dataset used is Task09_Spleen.tar from 
the [Medical Segmentation Decathlon](http://medicaldecathlon.com/). 
Prior to running this notebook the data should be downloaded following 
the step 9 in [Download Decathlon DataSet Notebook](../Data/DownloadDecathlonDataSet.ipynb).


# Lets get started
Before we get started lets check that we have an NVIDIA GPU available in the docker by running the cell below. 


In [ ]:
# following command should show all gpus available 
!nvidia-smi

In [ ]:
# Run some imports and functions used in the notebook

# Import some of the libraries that we will use throughout this notebook.

# setting up MMAR root path
MMAR_ROOT="/claraDevDay/AIAA/deepgrow_2D/"
AIAA_PORT="5000"
print ("MMAR_ROOT is set to ",MMAR_ROOT)
!ls $MMAR_ROOT
!chmod 777 $MMAR_ROOT/commands/*

from os import listdir
from os.path import isfile, join
from IPython.display import Image

# 1. Train Deep grow 2D  
 
### 1.1 Pre-processing the dataset
Deep grow needs to generate foreground and background user clicks 
as well as removing slices without any labels. 
For this we need to run a pre-processing script   


In [ ]:
tmpDataDir="/claraDevDay/Data/tmp4Deepgrow_2DDataPrep/"
dataset_root=""
dataset_json=""
# uncomment lines below if you are running the End2End notebook workflow
#dataset_root="/claraDevDay/Data/NSCLC_5/nii/"
#dataset_json=dataset_root+"dataSet.json"

! $MMAR_ROOT/commands/prepare_dataset.sh $tmpDataDir $dataset_root $dataset_json

Now lets check on the generated numpys

In [ ]:
!ls $tmpDataDir
!ls $tmpDataDir/images
!ls $tmpDataDir/labels


### 1.2 Train deep grow 2D
The pre-processing step prepares the data and also generate the `dataset.json` file needed for training 
Now we need to adjust the `enviroment.json` then run the `train.sh`  

In [ ]:
! $MMAR_ROOT/commands/train.sh 2


### 1.3 Check on results be running validation


In [ ]:
! $MMAR_ROOT/commands/validate.sh


lets see the results of our model on the validation set.
first we will make list of images then display them

In [ ]:
evalImageDir=MMAR_ROOT+'eval/segs/'
!ls $evalImageDir
onlyfiles = [f for f in listdir(evalImageDir) if isfile(join(evalImageDir, f))]

Now lets see the results 

In [ ]:
for f in onlyfiles:
    display(Image(filename=evalImageDir+f) )

### 1.4 Load model into AIAA 
####  1.4.1 Export the model
Training with this small set we can obtain a dice of 0.9. This is a great start.
Now we need to convert the model into AIAA

In [ ]:
! $MMAR_ROOT/commands/export.sh

#### 1.4.2 Load model into AIAA 

In [ ]:
modelName="MySpleenDeepgrow_2D"
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
config_str=' -F config=@"'+MMAR_ROOT+'/config/config_aiaa.json;type=application/json"'
model_str=' -F data=@"'+MMAR_ROOT+'/models/model.ts"'
cmd="curl -X PUT "+http_str+config_str+model_str
print ("running cmd = "+cmd)
! $cmd

No lets check that the model is loaded correctly into AIAA by checking the available models on the AIAA


In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/v1/models"
!curl -X GET $http_str -H "accept: application/json"


#### 1.5 Test with AIAA client  
You should now be able to test the mode using one of the AIAA Clients as shown in [AIAA Clients](AIAA.ipynb#AIAA-Clients)

# 2. Train Deep grow 3D
We will repeat the above steps but only changing the root mmar to point to the 3d deep grow mmar 

In [ ]:
MMAR_ROOT="/claraDevDay/AIAA/deepgrow_3D/"
print ("MMAR_ROOT is set to ",MMAR_ROOT)
!ls $MMAR_ROOT
!chmod 777 $MMAR_ROOT/commands/*
 

### 2.1 Prepare the data 

In [ ]:
tmpDataDir="/claraDevDay/Data/tmp4Deepgrow_3DDataPrep/"
dataset_root=""
dataset_json=""
# uncomment lines below if you are running the End2End notebook workflow
#dataset_root="/claraDevDay/Data/NSCLC_5/nii/"
#dataset_json=dataset_root+"dataSet.json"

! $MMAR_ROOT/commands/prepare_dataset.sh $tmpDataDir $dataset_root $dataset_json


Now lets check on the produced numpy files

In [ ]:
!ls $tmpDataDir
!ls $tmpDataDir/images
!ls $tmpDataDir/labels


### 2.2 Train deep grow 3D

In [ ]:
! $MMAR_ROOT/commands/train.sh 2


### 2.3 Check on results be running validation


In [ ]:
! $MMAR_ROOT/commands/validate.sh


lets see the results of our model on the validation set.
first we will make list of images then display them

In [ ]:
evalImageDir=MMAR_ROOT+'eval/segs/'
!ls $evalImageDir

You can use any 3d viewer as itksnap to visualize these volume.
  

### 2.4 Load model into AIAA 
####  2.4.1 Export the model
Training with this small set we can obtain a dice of ___. This is a great start.
Now we need to convert the model into AIAA

In [ ]:
! $MMAR_ROOT/commands/export.sh

#### 2.4.2 Load model into AIAA 

In [ ]:
modelName="MySpleenDeepgrow_3D"
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
config_str=' -F config=@"'+MMAR_ROOT+'/config/config_aiaa.json;type=application/json"'
model_str=' -F data=@"'+MMAR_ROOT+'/models/model.ts"'
cmd="curl -X PUT "+http_str+config_str+model_str
print ("running cmd = "+cmd)
! $cmd

No lets check that the model is loaded correctly into AIAA by checking the available models on the AIAA

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/v1/models"
!curl -X GET $http_str -H "accept: application/json"


#### 2.5 Test with AIAA client  
You should now be able to test the mode using one of the AIAA Clients as shown in [AIAA Clients](AIAA.ipynb#AIAA-Clients)



# 3. Creating a pipeline from 2D and 3D deep grow models
A new feature was added which allowed to create a pipeline that triggers multiple models. 
We have created a pipeline that would trigger 3D deepgrow model to get an inital segmentation, 
then use the center at each slice to trigger multiple 2D deep grow. 

Cell below adds the pipeline to the AIAA server  

In [ ]:
modelName="MySpleenDeepgrow_Pipeline"
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
config_str=' -F config=@"'+MMAR_ROOT+'/config/deepgrow_pipeline.json;type=application/json"'
cmd="curl -X PUT "+http_str+config_str
print ("running cmd = "+cmd)
! $cmd


In your AIAA client (Slicer or OHIF) you can refresh the models to see the pipeline.
Sine pipeline can contain any models, pipelines would appear in the segmentation and deepgrow section
